# Imports

In [1]:
import sys
sys.path.append("../libs/")
from data_prep import *
from ML_functions import *


# Data preprocessing for the appendix dataset that I will use

In [2]:
########### RUN THIS CELL ONLY ONCE !


##Tests 
def test1(value): ## Test to group by age
    if value>=35:
        return 1
    else:
        return 0


def tested(value): ## Test to group by education
    if value <=2 :
        return 1
    else:
        return 0
###

# Specify the file path
file_path = "../Dataframes/Appendix_dataset/defaults.xls"

# Read the Excel file into a DataFrame and do some preprocessing
df = pd.read_excel(file_path, index_col=0)
print(df.columns)
df=apply_function_to_column(df=df,column_name="X5",test_function=test1, new_name="Age_group") 
df=apply_function_to_column(df=df,column_name="X3",test_function=tested, new_name="Education_group") 
y = df["Y"].to_frame().copy()
num_features=["X1", "X5"] + [f"X{i}" for i in range (12,24,1)] +["Y"] #Y should not be dummied
cat_features=[col_name for col_name in df.columns if col_name not in num_features]

df["X2"]=df["X2"]-1

df = pd.get_dummies(df, columns = cat_features, drop_first= True)
X = df.drop("Y", axis=1).copy()




## Do train test split to get train val and test sets. 
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.4, random_state=123)
X_val,X_test,y_val,y_test=train_test_split(X_test,y_test, test_size=0.5, random_state=123) ##this make 0.2 for both val and test


## Save sensitive attributes

gender_train=X_train["X2_1"]
age_train=X_train["Age_group_1"]

gender_test=X_test["X2_1"]
age_test=X_test["Age_group_1"]

gender_val=X_val["X2_1"]
age_val=X_val["Age_group_1"]


education_train=X_train["X3_1"]
education_test=X_test["X3_1"]
education_val=X_val["X3_1"]

#Drop columns for without attribute and redundant ones

X_train_with_A=X_train.copy() #X with sensitive_attributes
X_test_with_A=X_test.copy() #X with sensitive_attributes
X_val_with_A=X_val.copy() #X with sensitive_attributes






Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21',
       'X22', 'X23', 'Y'],
      dtype='object')


# Store the processed data

In [3]:
store_data(dataset_name='Appendix_dataset',X_train_with_A=X_train_with_A,  X_val_with_A=X_val_with_A, X_test_with_A = X_test_with_A, y_train =  y_train,
            y_val = y_val, y_test = y_test, age=[age_train, age_val, age_test],
            gender= [gender_train, gender_val, gender_test], education=[education_train, education_val, education_test])


Dataframes saved in their directory from 'Dataframes' directory.


# Find and save best model for future use

In [4]:

# I kept commented the other models just to show that I tried more models in the beginning


####SCALE
X_train_with_A, X_val_with_A, X_test_with_A = scale_dataframes(
    [X_train_with_A, X_val_with_A, X_test_with_A])  ###scale all dfs ##Take care scale keeps 0,1 true


############# find best model

#param_grid_rf = {
    #'n_estimators': [10,50, 100],
    #'max_depth': [5,10],
    #'min_samples_leaf': [ 8,16]
#}


#param_grid_svc = {
    #'C': [0.1, 1, 10],
    #'kernel': ['linear']
#}

#param_grid_knn = {
    #'n_neighbors': [3,5, 10, 20]
#}


param_grid_lr = {
    'C': [ 0.001,0.01,0.1,1],
    'penalty': ['l2',
    ]
}




#model_rf=RandomForestClassifier()
#model_knn = KNeighborsClassifier()
model_lr = LogisticRegression(max_iter=500)  



#best_rf_A = find_best_model(model_rf, param_grid_rf, X_train_with_A, y_train.values.ravel(),  X_val_with_A, y_val.values.ravel())
#best_knn_A = find_best_model(model_knn, param_grid_knn, X_train_with_A, y_train.values.ravel(), X_val_with_A, y_val.values.ravel() )
best_lr_A = find_best_model(model_lr, param_grid_lr, X_train_with_A, y_train.values.ravel(),  X_val_with_A, y_val.values.ravel())


dataset_name="Appendix_dataset"
# Define the directory path for saving models
models_directory = os.path.join( '..', 'ML_models', dataset_name)  # '..' moves one directory up


if not os.path.exists(models_directory):
    os.makedirs(models_directory)

# Save the best models to separate files
#joblib.dump(best_rf_A, os.path.join(models_directory, 'best_random_forest_A_model.pkl'))
#joblib.dump(best_knn_A, os.path.join(models_directory, 'best_knn_A_model.pkl'))
joblib.dump(best_lr_A, os.path.join(models_directory, 'best_logistic_regression_A_model.pkl'))



Best Model Hyperparameters: {'C': 0.1, 'penalty': 'l2'}
Validation Accuracy: 0.8211666666666667


['..\\ML_models\\Appendix_dataset\\best_logistic_regression_A_model.pkl']